In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import matplotlib 
from PIL import Image
import PIL
import math
from random import random
from random import seed
from random import randint
import time
from numba import jit
import skimage

In [8]:
#numpy.roll
#from numba import jit

@jit(nopython = True)
    
def Laplacian(A,deltax,deltat):

    h = len(A)
    #CFL condition delta x => sqrt(D) delta t, where D is the number of dimensions            
    P = np.zeros((h,h))
    result = np.zeros((h,h))
    
    for i in range(0, h):
        for j in range(0,h):
            P[i,j] = (-A[(i+2)%h,j] + 16*A[(i+1)%h,j] + 16*A[(i-1)%h,j] -A[(i-2)%h,j] \
                 -A[i,(j+2)%h] + 16*A[i,(j+1)%h]  + 16*A[i,(j-1)%h] -A[i,(j-2)%h] - 60*A[i,j])/(12*pow(deltax,2))

  
    return P 

def damping(t,K_damp):
    Damping = np.zeros(shape=int(t/deltat))
    Damping[0:200] = 1*K_damp
    return Damping

def div_potential(A,omega,eta):
    #omega = 15
    lam = 2*pow(math.pi,2)/pow(omega,2)
    pot = lam*A*(pow(A,2)-pow(eta,2))
    return pot
    
def function(A,vel,omega,eta,deltax,deltat,index,Damping):
    F = np.subtract(Laplacian(A,deltax,deltat),div_potential(A,omega,eta)) - Damping[int(index)]*vel
    return F

def update(N,A,vel,omega,eta,deltax,deltat,index,Damping):
    F = function(A,vel,omega,eta,deltax,deltat,index,Damping)
    A = A + deltat*(vel + 0.5*deltat*function(A,vel,omega,eta,deltax,deltat,index,Damping)) 
    vel = vel + deltat/2*(F + function(A,vel,omega,eta,deltax,deltat,index,Damping))
    return A,vel
    
#change the values accordlingly    
def wolff(N,t,A,vel,omega,eta,deltax,deltat,K_damp):
    Damping = damping(t,K_damp)

    
    for i in np.arange(int(t/deltat)):
        plt.imsave(str(round((i*deltat),1))+'.png',A,vmin = -eta, vmax = eta,cmap = 'Greys')
        plt.imsave(str(round((i*deltat),1))+'a.png',pow(A,2),vmin = 0, vmax = eta,cmap = 'Greys')
        
        A,vel = update(N,A,vel,omega,eta,deltax,deltat,i,Damping)

        

    return A




In [9]:
start = time.time()


K_damp = 1
n = 7
deltax = 1
deltat = 0.1
eta = 1
N = pow(2,n)
t = int(0.5*N*deltax/deltat)
vel = np.zeros((N,N))
mu, sigma = 0, 0.1 # mean and standard deviation
omega = 5

A = np.random.normal(mu, sigma, size=(N, N))

A = wolff(N,t,A,vel,omega,eta,deltax,deltat,K_damp)

end = time.time()
print(end - start)

91.56630182266235


In [ ]:
"""        im = Image.fromarray(A)
    
        size = tuple((np.array(im.size) * 4).astype(int))
        c = np.array(im.resize(size, PIL.Image.BICUBIC))

        plt.imsave(str(round((i*deltat),1))+'.png',c,vmin = -eta, vmax = eta,cmap = 'Greys')"""
        

In [37]:
def Laplacian(A,deltax,deltat):

    #CFL condition delta x => sqrt(D) delta t, where D is the number of dimensions           

    
    Ax1 = np.roll(A,1,axis=0)
    Ax2 = np.roll(A,2,axis=0)
    Ax_1 = np.roll(A,-1,axis=0)
    Ax_2 = np.roll(A,-2,axis=0)
    
    Ay1 = np.roll(A,1,axis=1)
    Ay2 = np.roll(A,2,axis=1)
    Ay_1 = np.roll(A,-1,axis=1)
    Ay_2 = np.roll(A,-2,axis=1)
    
    P = (-Ax2 + 16*Ax1 + 16*Ax_1 - Ax_2 - Ay2 + 16*Ay1 + 16*Ay_1 - Ay_2 - 60*A)/(12*pow(deltax,2))
    
  
    return P


#numpy.roll


start = time.time()

n = 7
deltax = 1
deltat = 0.1
eta = 1
N = pow(2,n)
t = 0.5*N*deltax/deltat
vel = np.zeros((N,N))
mu, sigma = 0, 0.1 # mean and standard deviation
omega = 5

A = np.random.normal(mu, sigma, size=(N, N))

A = wolff(N,t,A,vel,omega,eta,deltax,deltat,K_damp)

end = time.time()
print(end - start)

56.127275228500366


In [ ]:
#change the values accordlingly    
def wolfff(N,t,A,vel,omega,eta,deltax,deltat,K_damp):
    damping = damping(t,K_damp)

    for i in np.arange(t/deltat):

        A,vel = update(N,A,vel,omega,eta,deltax,deltat,i)
        #fig = plt.figure(figsize=(10,10))
        fig = plt.figure()
        plt.imshow(A, vmin= -round(eta), vmax=round(eta),cmap = 'Greys', interpolation = 'none')
        fig.savefig(str(round((i*deltat),1))+'.png',bbox_inches = 'tight')
        plt.close(fig)
    return A



In [31]:
len(np.arange(t/deltat))

6400